# Redes Neurais Recorrentes

Long-Short Term Memory (LSTM) são um tipo de rede neural, mais especificamente um tipo de rede neural recorrente.

Redes neurais recorrentes possuem uma característica distinta de redes neurais feedforward. Ao invés da informação seguir um fluxo contínuo sempre em um direção (usualmente para "frente"), as RNNs passam a informação também de volta ("trás"). Isso permite que essas simulem uma memória, sendo capazes de lidar melhor com problemas que variam com o tempo, como é o nosso caso.

In [1]:
import torch
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from context import fakenews
from fakenews import preprocess as pre
import gensim
import math

In [2]:
base = '/home/thales/dev/fakenews/'
news, labels = pre.run(base + 'data/Fake.csv', base + 'data/True.csv')
pre.truncate_news(news)
splits = train_test_split(news, labels, test_size=0.3, shuffle=True)
news_trn, news_tst, labels_trn, labels_tst = splits

Succesfully read data from:
Fakes: /home/thales/dev/fakenews/data/Fake.csv
Reals: /home/thales/dev/fakenews/data/True.csv
Removing rows without text...
Removing publisher information...
Adding class column...
Merging fakes and reals
Merging titles and bodies...
Removing subjects and date...
Tokenizing data...
Truncating at 869


In [3]:
w2v = gensim.models.Word2Vec.load('fakenews-w2v.model')

In [4]:
w2v_as_tensor = torch.from_numpy(w2v.wv[news[0]])
w2v_as_tensor

tensor([[-1.0627, -1.2481, -0.9354,  ...,  2.8253, -3.2094,  1.8440],
        [ 0.1891, -0.6127,  1.1100,  ..., -1.1247,  0.3503,  0.4634],
        [ 0.4280,  0.7700, -0.9733,  ...,  0.3568, -1.1510, -0.2338],
        ...,
        [ 0.0471,  0.2390,  0.3499,  ..., -0.3489,  0.0388,  0.1113],
        [-2.3564,  0.9867,  5.5007,  ..., -3.6863, -1.0067,  1.9773],
        [-0.2077,  1.3571,  2.8597,  ..., -2.8318, -1.2677,  1.8112]])

In [5]:
w2v_as_tensor.shape

torch.Size([302, 100])

In [6]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din + dh, dout)

In [7]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din + dh, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.func = torch.sigmoid

In [25]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din + dh, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.func = torch.sigmoid
        
    def forward(self, X, hidden):
        xt_ht = torch.cat((X, hidden), 0)
        hidden = self.func(self.input(xt_ht))
        output = self.hidden(hidden)
        return self.func(output), hidden

In [26]:
criterion = torch.nn.functional.mse_loss

In [27]:
def train(model, X, y, optim, nepochs):
    niter = 0
    for epoch in range(nepochs):
        hidden = torch.zeros(50)
        for new, lbl in zip(X, y):
            for word in w2v.wv[new]:
                word_tensor = torch.from_numpy(word)
                output, hidden = model(word_tensor, hidden)
            loss = criterion(output, lbl)
            print(f"{epoch} Loss = {loss}")
            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for param in model.parameters():
                    param.data.add_(param.grad.data, alpha=-1e-5)
            niter += 1
            if niter == 10:
                break

In [28]:
elman_rnn = ElmanRNN(100, 50, 1)
sgd = torch.optim.SGD(elman_rnn.parameters(), lr=1e-6)
y = torch.tensor(labels_trn.values).view(len(labels_trn), 1)
train(elman_rnn, news_trn, y.type(torch.float), sgd, 1)

0 Loss = 0.39281293749809265
0 Loss = 0.4038773775100708


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

### LSTM

Como vimos na parte teórica, a LSTM nada mais é que uma variação do "neurônio" de uma RNN.

![lstmhidden](imgs/lstmhidden.png)

O PyTorch também possui uma classe para o LSTM, tendo em vista sua popularidade.

In [ ]:
class LSTMFNews:
    
    def __init__(self, input_dim, hidden_dim, optim):
        self.embed = w2v
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim)
        self.output = torch.nn.Linear(hidden_dim, 1)
        
    def forward(data, targets, nepochs=100):
        tensordata = torch.TensorDataset(data, targets)
        dtl = DataLoader(tensordata)
        hidden = torch.rand(hidden_dim)
        
        vec = torch.tensor(self.embed.wv[data])
        out, hidden = self.lstm(vec.view(1, 1, -1), hidden)
        self.output()
                    
                

Na instância acima, definimos uma LSTM onde o *hidden state* possui dimensão $M\times N\times S$. Uma característica específica da LSTM nesse framework é a entrada esperada e o estado. Ambos devem ser tensores tridimensionais. Podemos treinar a rede ao propagar um elemento da sequência por vez

In [ ]:
# treina 1 em 1

Enquanto que para usar todos os elementos precisamos concatenar nossos dados.

In [ ]:
# todos de uma vez

Principalmente em NLP, a transformação do texto para representação numérica é bastante comum. Dessa forma, podemos criar uma LSTM customizada ao adicionar um layer de embedding que fica responsável por essa transformação.

In [ ]:
class LSTMfnews:
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Assim como podemos usar o novo layer durante o treinamento, aprendendo os embeddings de acordo com a task.

In [ ]:
# treina embedding + lstm

Claro que também podemos criar vários franksteins, da mesma forma que fizemos com a RNN. Note que o *gradient descent* é totalmente genérico. Isto é, ele não depende da arquitetura do seu modelo, apenas que as funções de ativação sejam deriváveis ou deriváveis por partes. Por isso podemos criar estruturas totalmente **idiotas**, com valores indo, voltando, pulando, ignorando, com muito layer, e com o que quisermos.

In [ ]:
# cria frankstein

É importante notar que redes profundas tendem a ocasionar *overfitting*, assim como os problemas decorrentes do algoritmo de aprendizado:

* Vanishing
* Exploding